In [94]:
import csv
import time

def load_data(user_item_file):
    user_item_data = {}
    with open(user_item_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        
        for row in reader:
            user_id = row[0]
            item_id = row[1]
            if user_id in user_item_data:
                user_item_data[user_id].append(item_id)
            else:
                user_item_data[user_id] = [item_id]
    
    return user_item_data


In [ ]:
# HashTable class definition
class HashTable:
    def __init__(self, size, collision_avoidance):
        self.table = [[] for _ in range(size)] if collision_avoidance == 'separate_chaining' else [None] * size
        self.size = size
        self.collisions = 0
        self.collision_avoidance = collision_avoidance

    def hash(self, key):
        return hash(key) % self.size

    def second_hash(self, key):
        return 1 + (hash(key) % (self.size - 1))

    def insert(self, key, value):
        if self.collision_avoidance == 'separate_chaining':
            # Separate Chaining Implementation
            index = self.hash(key)
            
            # Check if key already exists in the chain
            for i, (k, v) in enumerate(self.table[index]):
                if k == key:
                    self.table[index][i] = (key, value)  # Update existing
                    return
            
            # If chain is not empty we have a collision
            if len(self.table[index]) > 0:
                self.collisions += 1
            
            # Add new key-value pair to the chain
            self.table[index].append((key, value))
        
        else:  # Linear Probing
            index = self.hash(key)
            probe = 0
            
            while probe < self.size:
                new_index = (index + probe) % self.size
                
                # Empty slot found
                if self.table[new_index] is None:
                    self.table[new_index] = (key, value)
                    return
                
                # Key already exists, update it
                if self.table[new_index][0] == key:
                    self.table[new_index] = (key, value)
                    return
                
                # Collision - slot is occupied, try next
                self.collisions += 1
                probe += 1
            
            # Table is full
            raise Exception("Hash table is full")

    def retrieve(self, key):
        if self.collision_avoidance == 'separate_chaining':
            # Separate Chaining Implementation
            index = self.hash(key)
            
            # Search through the chain
            for k, v in self.table[index]:
                if k == key:
                    return v
            
            return None  # Key not found
        
        else:  # Linear Probing
            index = self.hash(key)
            probe = 0
            
            while probe < self.size:
                new_index = (index + probe) % self.size
                
                # Empty slot means key doesn't exist
                if self.table[new_index] is None:
                    return None
                
                # Key found
                if self.table[new_index][0] == key:
                    return self.table[new_index][1]
                
                probe += 1
            
            return None  # Key not found

In [96]:
class MaxHeap:
    def __init__(self):
        self.heap = []

    def parent(self, index):
        return (index - 1) // 2

    def left_child(self, index):
        return 2 * index + 1

    def right_child(self, index):
        return 2 * index + 2

    def has_left(self, index):
        return self.left_child(index) < len(self.heap)

    def has_right(self, index):
        return self.right_child(index) < len(self.heap)

    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def percolate_up(self, index):
        parent = self.parent(index)
        if index > 0 and self.heap[index][0] > self.heap[parent][0]:
            self.swap(index, parent)
            self.percolate_up(parent)

    def percolate_down(self, index):
        largest = index
        left = self.left_child(index)
        right = self.right_child(index)

        if self.has_left(index) and self.heap[left][0] > self.heap[largest][0]:
            largest = left
        if self.has_right(index) and self.heap[right][0] > self.heap[largest][0]:
            largest = right
        if largest != index:
            self.swap(index, largest)
            self.percolate_down(largest)

    def push(self, priority, item):
        self.heap.append((priority, item))
        self.percolate_up(len(self.heap) - 1)

    def pop(self):
        if len(self.heap) > 1:
            self.swap(0, len(self.heap) - 1)
            item = self.heap.pop()
            self.percolate_down(0)
        else:
            item = self.heap.pop()
        return item

    def top_n(self, n):
        top_items = []
        for _ in range(min(n, len(self.heap))):
            top_items.append(self.pop()[1])
        return top_items


In [97]:
# RecommendationSystem class
class RecommendationSystem:
    def __init__(self, user_item_data):
        self.user_item_data = user_item_data

        # Store hash tables for different techniques
        self.hash_tables = {}  
    
    def build_recommendation_system(self, technique, size=20):
        # Create hash table
        hash_table = HashTable(size, technique)

        # Track insertion time
        start_time = time.time()

        # Insert all user-item pairs
        for user, items in self.user_item_data.items():
            hash_table.insert(user, items)
        end_time = time.time()
        insertion_time = end_time - start_time

        # Store hash table for this technique
        self.hash_tables[technique] = hash_table
        return hash_table, insertion_time
    
    def recommend_items_with_jaccard(self, target_user, technique, top_n=5):
        # Get hash table for this technique
        hash_table = self.hash_tables.get(technique)
        if hash_table is None:
            raise ValueError(f"Hash table for {technique} not built yet")
        
        # Track retrieval time
        start_time = time.time()

        # Get target user's items
        target_items = hash_table.retrieve(target_user)
        if target_items is None:
            return [], 0 
        target_items_set = set(target_items)

        # Compute Jaccard similarity with all other users
        similarity_by_user = {}
        for user in self.user_item_data.keys():
            if user == target_user:
                continue
            other_items = hash_table.retrieve(user)
            if other_items is None:
                continue
            other_items_set = set(other_items)
            
            # Calculate Jaccard similarity
            intersection = len(target_items_set & other_items_set)
            union = len(target_items_set | other_items_set)
            
            if union > 0:
                similarity = intersection / union
                similarity_by_user[user] = similarity
        
        # Accumulate item scores from similar users
        item_scores = {}
        
        for user, similarity in similarity_by_user.items():
            user_items = hash_table.retrieve(user)
            
            for item in user_items:
                # Only recommend items the target user hasn't purchased
                if item not in target_items_set:
                    if item in item_scores:
                        item_scores[item] += similarity
                    else:
                        item_scores[item] = similarity
        
        # Build MaxHeap with scored items
        recommendations_heap = MaxHeap()
        
        for item, score in item_scores.items():
            recommendations_heap.push(score, item)
        
        # Get top N recommendations
        top_recommendations = recommendations_heap.top_n(top_n)
        
        end_time = time.time()
        retrieval_time = end_time - start_time
        
        return top_recommendations, retrieval_time


In [98]:
def main():
    user_data = load_data('user_item_data.csv')
    
    # Create recommendation system 
    rec_system = RecommendationSystem(user_data)
    
    # techniques to test
    techniques = ['separate_chaining', 'linear_probing']
    table_size = 7
    
    # Build hash tables for both techniques
    results = {}
    
    for technique in techniques:
        hash_table, insertion_time = rec_system.build_recommendation_system(technique, table_size)
        results[technique] = {
            'hash_table': hash_table,
            'insertion_time': insertion_time,
            'collisions': hash_table.collisions
        }
    
    # Generate recommendations for each user with both techniques
    top_n = 3  # Number of recommendations per user
    
    for technique in techniques:
        print(f"Using collision avoidance technique: {technique}")
        
        hash_table = results[technique]['hash_table']
        
        # Get recommendations for each user
        for user in sorted(user_data.keys()):
            recommendations, retrieval_time = rec_system.recommend_items_with_jaccard(
                user, technique, top_n
            )
            
            print(f"Recommendations for {user}: {recommendations}")
        
        print(f"Insertion Time: {results[technique]['insertion_time']:.6f} seconds")
        print(f"Retrieval Time: {retrieval_time:.6f} seconds")
        print(f"Collisions: {results[technique]['collisions']}")
        print()

if __name__ == "__main__":
    main()


Using collision avoidance technique: separate_chaining
Recommendations for user1: ['The Last of Us', 'Battlefield', 'Resident Evil']
Recommendations for user2: ['Bayonetta', 'Splatoon', 'Minecraft']
Recommendations for user3: ['Bayonetta', 'Resident Evil', 'League of Legends']
Recommendations for user4: ['Silent Hill', 'Half-Life', 'Splatoon']
Recommendations for user5: ['Resident Evil', 'Portal', 'Bayonetta']
Recommendations for user6: ['The Legend of Zelda', 'Dragon Age', 'Spider-Man']
Recommendations for user7: ['Animal Crossing', 'The Legend of Zelda', 'Bioshock']
Insertion Time: 0.000020 seconds
Retrieval Time: 0.000069 seconds
Collisions: 2

Using collision avoidance technique: linear_probing
Recommendations for user1: ['The Last of Us', 'Battlefield', 'Resident Evil']
Recommendations for user2: ['Bayonetta', 'Splatoon', 'Minecraft']
Recommendations for user3: ['Bayonetta', 'Resident Evil', 'League of Legends']
Recommendations for user4: ['Silent Hill', 'Half-Life', 'Splatoon']
R

Comment Analysis

Separate Chaining: 2 collisions
Linear Probing: 7 collisions

Separate chaining counts collisions only when a new user hashes to a slot that already has users. With 7 users distributed across 7 slots, some slots remain empty while others receive multiple users. The 2 collisions indicate that 2 users were added to slots that already had one user.

Linear probing counts every occupied slot encountered while searching for an empty position. As the table fills up, later insertions must probe through multiple occupied slots, resulting in more collisions (7 total).



Separate chaining
Strengths: Handles high load factors well, never runs out of space, simpler collision counting
Weaknesses: Requires extra memory for linked lists, slower if chains get long

Linear probing
Strengths: It uses less memory than seperate chaining because it stores all elements in a array
Weaknesses: performance goes down as table fills up, table can become full

Why MaxHeap is Best
The MaxHeap extracts the top N recommendations by maintaining items in priority order. With O(log n) insertion and O(log n) extraction, it's faster than sorting all items when we only need the top few. This is important for real-time recommendation systems.